<a href="https://www.nb.no/" style="display: flex; align-items: center;">
    <img src="./NB_logo_navy.png" alt="Nasjonalbiblioteket" width="150" style="margin-right: 10px;" />
    <h1 style="margin: 0;">Nasjonalbiblioteket</h1>
</a>

# Last inn bilder fra Nettbiblioteket til Transkribus

In [ ]:
import requests

import handskrift as hs

## Logg inn

Legg inn ditt Transkribus-brukernavn og passord.

In [ ]:
# OBS: Ikke del filen med brukernavn og passord

USERNAME = "BRUKERNAVN"
PASSWORD = 'PASSWORD'

In [ ]:
# Opprett forbindelse til Transkribus. Denne er åpen i et par timer.

my_session = requests.Session()
login_response = my_session.post(
    "https://transkribus.eu/TrpServer/rest/auth/login", data={"user": USERNAME, "pw": PASSWORD}
)

print(
    "Du er logget inn!"
    if login_response.ok
    else "OBS! Kunne ikke logge inn. Sjekk påloggingsinfoen og prøv igjen."
)

## Finn fram dokumenter

Fyll inn søkeparametere i `query` for å finne dokumenter fra Nettbiblioteket ([nb.no](https://www.nb.no/search?mediatype=brev-og-manuskripter)).


In [ ]:
query = {
    "filter": [
        "mediatype:privatarkivmateriale",
        'api_title:"TSarkA 2"',
    ],
    "digitalAccessibleOnly": "true",
    "size": 10,  # max 50
    "fragments": 2,
    "profile": "nbdigital",
    "scroll": True,
}

sesamids = hs.run_query(query)

In [ ]:
# Se hvilke id-er vi har hentet
print("Antall sesam-IDer: ", len(sesamids))
print("Sesam-IDer: ")
print("\n".join(sesamids))

## Sammenlign listen med ID-ene som allerede ligger i samlingen i Transkribus

In [ ]:
# Fyll inn navnet på Transkribus-samlingen du vil laste opp til
collection_name = ""

In [ ]:
# Hent ID-en til samlingen
coll_id = hs.get_collection_id(collection_name, my_session)

In [ ]:
# Hent dokument-ID-ene i Transkribus-samlingen
transkribus_ids = hs.get_sesamids_from_transkribus_collection(coll_id, my_session)

# Filtrer ID-ene vi har hentet fra Nettbiblioteket mot ID-ene vi har i Transkribus
new_docs = hs.filter_sesamids(sesamids, transkribus_ids)
print(f"Antall nye dokumenter fra Nettbiblioteket: {len(new_docs)}")

# Lagre til fil
hs.save_sesamids(new_docs, "docs_to_upload.txt")

### Last opp fra NB til Transkribus

OBS! Hvis det er flere enn 10 dokumenter som skal lastes opp kommer det til å ta en stund.

Vi anbefaler heller å kjøre `handskrift.py` fra terminalen i en skjerm-uavhengig økt (med `screen`, `tmux` eller lignende).


```shell
tmux 
source .venv/bin/activate
python handskrift.py -c $COLLECTION_ID -u $USERNAME -p $PASSWORD -i docs_to_upload.txt > log.txt
```

In [ ]:
hs.lastopp_transkribus(collId=coll_id, s=my_session, sesamids=new_docs)

### Gå inn i Transkribus for å se på dokumentene

https://lite.transkribus.eu/
